In [34]:
!pip install clean-text

     |████████████████████████████████| 53 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 175 kB 6.2 MB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171047 sha256=7545ac26bf64a5c4f58710eae26aa318962820171458eaaaaf9572e7fc22a7ba
  Stored in directory: /Users/william/Library/Caches/pip/wheels/fa/7a/e9/22dd0515e1bad255e51663ee513a2fa839c95934c5fc301090
Successfully built emoji


In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import spacy
from spacy import displacy

from cleantext import clean

In [36]:
sci_tech_data = pd.read_csv('./sci_tech_1670979259.409489.csv')

/Users/william/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (17,21,34,63,68,112) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
sci_tech_data.shape

(8058, 128)

Below I drop duplicate rows.

In [38]:
sci_tech_data.drop_duplicates(subset = ['selftext','title'], inplace=True)
sci_tech_data.shape

(7480, 128)

Below I select the relevent rows that have tech or science as the subreddits and save the feature space, 'title' and the target, 'subreddit' to respective variables.

In [39]:
sci_tech_data['subreddit'].value_counts()

technology                4570
science                   2206
u_Ok-Technology-1912       142
u_Candid-Science-7189      132
u_MEGA-Technology          132
u_Gold-Science-2230         94
u_Away-Technology-4883      63
u_Any-Technology-9975       59
u_AVID-Technology           27
u_Content-Technology-7      19
u_Bugd-Technology            7
u_SecureAge-Technology       4
u_kretoss-technology         3
u_Slow-Technology-9949       3
u_Ok-Science-1826            2
u_FS-Technology              2
u_Glum-Science-6510          1
u_No-Technology-2687         1
u_Jaded-Technology-332       1
u_more-technology-00         1
u_Dear-Technology-6015       1
u_Then-Technology-9558       1
u_Fearless-Science-693       1
u_Fearless-Science-103       1
u_TheSweet-Science           1
u_MR-Technology              1
u_awesome-technology         1
u_visimens-technology        1
u_Ok-Science-9480            1
u_A-science-enthusiast       1
u_Edu-Technology             1
Name: subreddit, dtype: int64

In [6]:
sci_tech_data['subreddit_name_prefixed'].value_counts()

r/technology              4570
r/science                 2206
u/Ok-Technology-1912       142
u/Candid-Science-7189      132
u/MEGA-Technology          132
u/Gold-Science-2230         94
u/Away-Technology-4883      63
u/Any-Technology-9975       59
u/AVID-Technology           27
u/Content-Technology-7      19
u/Bugd-Technology            7
u/SecureAge-Technology       4
u/kretoss-technology         3
u/Slow-Technology-9949       3
u/Ok-Science-1826            2
u/FS-Technology              2
u/Glum-Science-6510          1
u/No-Technology-2687         1
u/Jaded-Technology-332       1
u/more-technology-00         1
u/Dear-Technology-6015       1
u/Then-Technology-9558       1
u/Fearless-Science-693       1
u/Fearless-Science-103       1
u/TheSweet-Science           1
u/MR-Technology              1
u/awesome-technology         1
u/visimens-technology        1
u/Ok-Science-9480            1
u/A-science-enthusiast       1
u/Edu-Technology             1
Name: subreddit_name_prefixed, dtype: i

In [7]:
[c for c in sci_tech_data.columns]

['Unnamed: 0',
 'subreddit',
 'selftext',
 'author_fullname',
 'gilded',
 'title',
 'link_flair_richtext',
 'subreddit_name_prefixed',
 'hidden',
 'pwls',
 'link_flair_css_class',
 'thumbnail_height',
 'top_awarded_type',
 'hide_score',
 'quarantine',
 'link_flair_text_color',
 'upvote_ratio',
 'author_flair_background_color',
 'total_awards_received',
 'media_embed',
 'thumbnail_width',
 'author_flair_template_id',
 'is_original_content',
 'secure_media',
 'is_reddit_media_domain',
 'is_meta',
 'category',
 'secure_media_embed',
 'link_flair_text',
 'score',
 'is_created_from_ads_ui',
 'author_premium',
 'thumbnail',
 'edited',
 'author_flair_css_class',
 'author_flair_richtext',
 'gildings',
 'post_hint',
 'content_categories',
 'is_self',
 'subreddit_type',
 'link_flair_type',
 'wls',
 'removed_by_category',
 'author_flair_type',
 'domain',
 'allow_live_comments',
 'suggested_sort',
 'url_overridden_by_dest',
 'view_count',
 'archived',
 'no_follow',
 'is_crosspostable',
 'pinned',


In [42]:
subreddit = sci_tech_data.loc[(sci_tech_data['subreddit'].isin(
                                ['technology','science']), 'subreddit')]
title = sci_tech_data.loc[(sci_tech_data['subreddit'].isin(
                                ['technology','science']),'title')]

In [9]:
X = title
y = subreddit
y.value_counts()

technology    4570
science       2206
Name: subreddit, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

[source: strip characters from string in series](https://stackoverflow.com/questions/13682044/remove-unwanted-parts-from-strings-in-a-column)
[source: remove punctuation](https://www.google.com/search?q=how+to+replace+punctuation+with+regular+expression+python&rlz=1C5CHFA_enUS983US983&oq=how+to+replace+punctuation+with+regular&aqs=chrome.1.69i57j33i160l2.10574j0j7&sourceid=chrome&ie=UTF-8#kpvalbx=_7SabY4OANaSs0PEP042roAM_32)

In [11]:
X_train

6145    Flocks of assembler robots show potential for ...
5655                can we use internet without internet?
4484                                               ✂️ waz
23      US scientists confirm ‘major breakthrough’ in ...
2174    Biofilm-forming microorganisms causing hospita...
                              ...                        
5324                                 Why IT projects fail
3002         The future of finance belongs to open source
2509    Students who repeat a grade feel like they do ...
1351     https://secureblitz.com/best-antivirus-for-2022/
7317    Donald Trump floating in a sea of feces, hyper de
Name: title, Length: 5082, dtype: object

Get rid of puctuation

In [46]:
punctuation_pattern = '[^\w\s]'
giveaway_words = ['science', 'technology']
url_pattern = '[A-Za-z0-9]+://[A-Za-z0-9%-_]+(/[A-Za-z0-9%-_])*(#|\\?)[A-Za-z0-9%-_&=]*'

remove_group = giveaway_words + [punctuation_pattern] + [url_pattern]

re.sub('science',repl='', string = X_train[0])


'Flying snakes as a model for robot design'

In [47]:
# for r in remove_group:
#     X_train_filtered = X_train.map(lambda x: re.sub(r,repl='', string = x)) #
# print((sum([len(s) for s in X_train])-sum([len(s) for s in X_train_filtered]))/sum([len(s) for s in X_train])
# )
# [s for s in X_train_filtered if s]

0.002660021118019616


['Flocks of assembler robots show potential for making larger structures',
 'can we use internet without internet?',
 '✂️ waz',
 'US scientists confirm ‘major breakthrough’ in nuclear fusion',
 'Biofilm-forming microorganisms causing hospital-acquired infections from intravenous catheter -- a wide range of clinical isolates were able to form biofilms, including Gram-positive strains, Gram-negative strains, and Candida albicans',
 'How is it job situation ? I am from Russia and we have many factors(including political) now it is difficult to find the work of developers. Is there a decline in vacancies and an increase in the number of unemployed in your country?',
 'E-bike batteries have caused 200 fires in New York: ‘Everyone’s scared’',
 'People don’t mate randomly – but the flawed assumption that they do is an essential part of many studies linking genes to diseases and traits',
 'Having in mind the history of tech as a whole, do you think computers and processing units such as GPUs, 

remove items from 

In [64]:
for r in remove_group:
    X_train_filtered = X_train.map(lambda x: clean(x, no_emoji=True,no_punct=True,no_digits=True, no_urls=True))
    X_train_filtered = X_train_filtered.map(lambda x: re.sub('(0|\||(<url>))',repl='',string = x))#
print((sum([len(s) for s in X_train])-sum([len(s) for s in X_train_filtered]))/sum([len(s) for s in X_train])
)
[s for s in X_train_filtered]

0.04721634001941931


['flocks of assembler robots show potential for making larger structures',
 'can we use internet without internet',
 'waz',
 'us scientists confirm major breakthrough in nuclear fusion',
 'biofilmforming microorganisms causing hospitalacquired infections from intravenous catheter a wide range of clinical isolates were able to form biofilms including grampositive strains gramnegative strains and candida albicans',
 'how is it job situation i am from russia and we have many factorsincluding political now it is difficult to find the work of developers is there a decline in vacancies and an increase in the number of unemployed in your country',
 'ebike batteries have caused  fires in new york everyones scared',
 'people dont mate randomly but the flawed assumption that they do is an essential part of many studies linking genes to diseases and traits',
 'having in mind the history of tech as a whole do you think computers and processing units such as gpus cpus and memory drives will get sma

In [15]:
#load medium pipeline
nlp = spacy.load('en_core_web_md')

I'll 

In [88]:
cv = CountVectorizer(stop_words=['science',''])
X_train_vec = cv.fit_transform(X_train_filtered)

(5082, 12892)

In [91]:
X_train_vec.A.

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [82]:
X_df.sum()

0    77156
dtype: int64

In [93]:
X_train_vec.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [94]:
X_df = pd.DataFrame(data = X_train_vec.todense(), columns = cv.get_feature_names_out())


In [107]:
X_df.sum().sort_values(ascending = False).keys()

Index(['the', 'to', 'of', 'and', 'in', 'for', 'is', 'on', 'with', 'that',
       ...
       'innoov', 'innermost', 'innately', 'inks', 'inked', 'injured', 'inizia',
       'initiative', 'initiate', 'zyple'],
      dtype='object', length=13008)

In [112]:
word_counts = pd.DataFrame({'word_count':X_df.sum().sort_values(ascending = False).values}, index = X_df.sum().sort_values(ascending = False).keys())
word_counts.shape

(13008, 1)

In [118]:
word_counts[word_counts['word_count']>5].shape

(2115, 1)

In [92]:
X_df = pd.DataFrame({'count':X_train_vec.sum().values}, index = X_train_vec.sum().keys())

AttributeError: 'numpy.int64' object has no attribute 'values'

In [20]:
CountVectorizer(stop_words=[])

CountVectorizer(stop_words=[])

In [59]:
X_df.sum().sort_values(A)

𝙞𝙨            1
iş            1
için          1
izotope       1
iyi           1
           ... 
in         1539
and        1601
of         1724
to         2281
the        2362
Length: 12892, dtype: int64

**unsure** if I clean the 

In [22]:
X_df.sum().sort_values(ascending = False)['technology']

118